In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LogisticRegression

In [ ]:
# making a Logistic Regression modeler from scratch using Numpy and linear algebra methods
class LogisticRegressionScratch(object):

    def __init__(self, tolerance = 10**-8, max_iterations = 20):

        self.tolerance = tolerance
        self.max_iterations = max_iterations
        self.weights_array = None # holds current weights and intercept (intercept is at the last position)
        self.prior_w = None # holds previous weights and intercept (intercept is at the last position)

        # once we are done training, these variables will hold the
        # final values for the weights and intercept
        self.weights = None
        self.intercept = None


    def predict_proba(self, X):
        '''
        Compute probabilities using the inverse logit
        - Inputs: The Nx(K+1) matrix with intercept column X
        - Outputs: Vector of probabilities of length N
        '''

        XW = np.dot(X, self.weights_array)
        P = 1 / (1+np.exp(-XW))
        return P

    def compute_gradient(self, X, Y, P):
        '''
        Computes the gradient vector
        -Inputs:
            - The Nx(K+1) matrix with intercept column X
            - Nx1 vector y (label)
            - Nx1 vector of predictions P
        -Outputs: 1x(K+1) vector of gradients
        '''

        G = -np.dot((Y - P), X)
        return G

    def compute_hessian(self, X, P):
        '''
        computes the Hessian matrix
        -inputs:
            - Nx(K+1) matrix X
            - Nx1 vector of predictions P
        -outputs:
            - KxK Hessian matrix H=X^T * Diag(Q) * X
        '''

        Q = P*(1-P)
        XQ = X.T * Q
        H = np.dot(XQ,X)
        return H


    def update_weights(self, X, y):
        '''
        Updates existing weight vector
        -Inputs:
            -Nx(Kx1) matrix X
            -Nx1 vector y
        -Calls predict_proba, compute_gradient and compute_hessian and uses the
        return values to update the weights array
        '''

        P = self.predict_proba(X)
        G = self.compute_gradient(X, y, P)
        H = self.compute_hessian(X, P)
        self.prior_w = self.weights_array
        H_inv = np.linalg.inv(H)
        self.weights_array = self.prior_w - np.dot(H_inv, G)


    def check_stop(self):
        '''
        check to see if euclidean distance between old and new weights (normalized)
        is less than the tolerance

        returns: True or False on whether stopping criteria is met
        '''

        w_old_norm = self.prior_w / np.linalg.norm(self.prior_w)
        w_new_norm = self.weights_array / np.linalg.norm(self.weights_array)
        diff = w_old_norm - w_new_norm
        distance = (np.dot(diff, diff))**0.5
        stop = distance < self.tolerance
        return stop


    def fit(self, X, y):
        '''
        X is the Nx(K-1) data matrix
        Y is the labels, using {0,1} coding
        '''

        #set initial weights - add an extra dimension for the intercept
        self.weights_array = np.zeros(X.shape[1] + 1)

        #Initialize the slope parameter to log(base rate/(1-base rate))
        self.weights_array[-1] = np.log(y.mean() / (1-y.mean()))

        #create a new X matrix that includes a column of ones for the intercept
        X_int = np.hstack((X, np.ones((X.shape[0],1))))

        for i in range(self.max_iterations):
            self.update_weights(X_int, y)

            # check whether we should
            stop = self.check_stop()
            if stop:
                # since we are stopping, lets save the final weights and intercept
                self.set_final_weights()
                self.set_final_intercept()
                break


    def set_final_weights(self):
        self.weights = self.weights_array[0:-1]

    def set_final_intercept(self):
        self.intercept = self.weights_array[-1]

    def get_weights(self):
        return self.weights

    def get_intercept(self):
        return self.intercept


In [ ]:
filename = os.path.join(os.getcwd(), "data", "airbnbData_train.csv")

In [ ]:
df = pd.read_csv(filename)

In [ ]:
feature_list = ['review_scores_rating','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_value','host_response_rate','host_acceptance_rate']
feature_list

['review_scores_rating',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_value',
 'host_response_rate',
 'host_acceptance_rate']

In [ ]:
X = df[feature_list]
y = df['host_is_superhost']

In [ ]:
lr = LogisticRegressionScratch()
lr.fit(X,y)

In [ ]:
print('The fitted weights and intercept are:')
print(lr.get_weights(), lr.get_intercept())

The fitted weights and intercept are:
[ 0.56690005  0.492255    0.201587    0.25551467 -0.00590516  1.71592957
  0.26478817] -1.829062262272181


In [ ]:
lr_sk = LogisticRegression(C=10**10)
lr_sk.fit(X,y)

LogisticRegression(C=10000000000, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
print('The fitted weights and intercept with sklearn are:')
print(lr_sk.coef_, lr_sk.intercept_)

The fitted weights and intercept with sklearn are:
[[ 0.56691547  0.49224905  0.20150113  0.25563246 -0.005929    1.71592022
   0.26479199]] [-1.82906726]


In [ ]:
%timeit(lr.fit(X,y))

28.1 ms ± 8.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%timeit(lr_sk.fit(X,y))

90.8 ms ± 5.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
# It seems that our logistic regression model is faster than the SciKit one.